# g1-4_v018
phones_mean　時間差大きい場合、平均の対象から外す

In [1]:
# import library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
from tqdm.notebook import tqdm
import pathlib
import plotly
import plotly.express as px
import itertools
import lightgbm as lgb
from optuna.integration import lightgbm as optuna_lgb
import simdkalman
import optuna
import pyproj
from pyproj import Proj, transform
from sklearn import metrics
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix, accuracy_score
pd.set_option('display.max_rows', 100)

In [2]:
g1 = ['2020-05-14-US-MTV-1', '2020-05-14-US-MTV-2', '2020-05-21-US-MTV-1', '2020-05-21-US-MTV-2',
      '2020-05-29-US-MTV-1', '2020-05-29-US-MTV-2', '2020-06-04-US-MTV-1', '2020-06-05-US-MTV-1',
      '2020-06-05-US-MTV-2', '2020-06-11-US-MTV-1', '2020-07-08-US-MTV-1', '2020-07-17-US-MTV-1',
      '2020-07-17-US-MTV-2', '2020-08-03-US-MTV-1', '2020-08-06-US-MTV-2', '2020-09-04-US-SF-1',
      '2020-09-04-US-SF-2',  '2021-01-04-US-RWC-1', '2021-01-04-US-RWC-2',
      '2020-05-15-US-MTV-1', '2020-05-28-US-MTV-1', '2020-05-28-US-MTV-2', '2020-06-04-US-MTV-2',
      '2020-06-10-US-MTV-1', '2020-06-10-US-MTV-2', '2020-08-03-US-MTV-2', '2020-08-13-US-MTV-1',
      '2021-03-16-US-MTV-2']

g2 = ['2021-01-05-US-SVL-1', '2021-01-05-US-SVL-2', '2021-04-15-US-MTV-1', 
      '2021-03-25-US-PAO-1', '2021-04-02-US-SJC-1', '2021-04-08-US-MTV-1']

g3 = ['2021-03-10-US-SVL-1', '2021-04-26-US-SVL-1', '2021-04-26-US-SVL-2']

g4 = ['2021-04-28-US-MTV-1', '2021-04-29-US-MTV-1', 
      '2021-03-16-US-RWC-2', '2021-04-21-US-MTV-1', '2021-04-28-US-MTV-2', '2021-04-29-US-MTV-2']

g5 = ['2021-04-22-US-SJC-1', '2021-04-28-US-SJC-1', '2021-04-29-US-SJC-2', 
      '2021-04-22-US-SJC-2', '2021-04-29-US-SJC-3']

In [3]:
target = g1 + g2 + g3 + g4

In [4]:
ro_th = 50 # 相対移動距離をもとにした異常値除去の閾値
rog_th = 10 # ground_truthをもとにした異常値除去の閾値

# ground_truthをもとにした異常値除去を行うcollection
rog_target = ['2021-04-22-US-SJC-1', '2021-04-29-US-SJC-2', '2021-04-28-US-SJC-1', '2021-04-22-US-SJC-2', '2021-04-29-US-SJC-3',
              '2021-04-28-US-MTV-1', '2021-04-29-US-MTV-1', '2021-03-16-US-RWC-2', '2021-04-28-US-MTV-2', '2021-04-29-US-MTV-2',
              '2021-04-26-US-SVL-2', '2021-03-10-US-SVL-1', '2021-04-26-US-SVL-1',
              '2021-04-21-US-MTV-1', '2021-04-28-US-MTV-1', '2021-04-29-US-MTV-1']

In [5]:
import ipynb_path

def get_nb_name():
    nb_path = ipynb_path.get()
    nb_name = nb_path.rsplit('/',1)[1].replace('.ipynb','')
    return nb_name

In [6]:
# directory setting
nb_name = get_nb_name()
INPUT = '../input/google-smartphone-decimeter-challenge'
OUTPUT = '../output/' + nb_name
os.makedirs(OUTPUT, exist_ok=True)

# utils

In [7]:
def get_train_score(df, gt):
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    # calc_evaluate_score
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    res = df.groupby('phone')['err'].agg([percentile50, percentile95])
    res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2 
    score = res['p50_p90_mean'].mean()
    return score

In [8]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [9]:
def visualize_trafic(df, center, zoom=9):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                            
                            zoom=zoom,
                            center=center,
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()
    
def visualize_collection(df, collection):
    target_df = df[df['collectionName']==collection].copy()
    lat_center = target_df['latDeg'].mean()
    lng_center = target_df['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
    
    visualize_trafic(target_df, center)

In [10]:
# ground_truth
def get_ground_truth():
    p = pathlib.Path(INPUT)
    gt_files = list(p.glob('train/*/*/ground_truth.csv'))

    gts = []
    for gt_file in gt_files:
        gts.append(pd.read_csv(gt_file))
    ground_truth = pd.concat(gts)

    return ground_truth

In [11]:
def percentile50(x):
    return np.percentile(x, 50)
def percentile95(x):
    return np.percentile(x, 95)

In [12]:
class train_result:
    def __init__(self, df):
        self.df = df
        self.gt = get_ground_truth()
        self.bl = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
        
        self.gt = self.gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
        self.df = self.df.merge(self.gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
        self.df['phone'] = self.df['collectionName'] + '_' + self.df['phoneName']
        self.df['err'] =  calc_haversine(self.df['latDeg_gt'], self.df['lngDeg_gt'], self.df['latDeg'], self.df['lngDeg'])
        
        self.phone_res = self.calc_err('phone')
        self.clc_res = self.calc_err('collectionName')
        self.phonename_res = self.calc_err('phoneName')
        
    def calc_err(self, by):
        res = self.df.groupby(by)['err'].agg([percentile50, percentile95])
        res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2
        return res
    
    @property
    def score(self):
        return self.phone_res['p50_p90_mean'].mean()
    @property
    def raw_data(self):
        return self.df
    @property
    def err(self):
        return self.phone_res
    @property
    def collection_err(self):
        return self.clc_res
    @property
    def phonename_err(self):
        return self.phonename_res
    
    def viz_map(self, collection, show_gt=True, show_bl=True):
        tmp = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp2 = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg_gt', 'lngDeg_gt']]
        tmp2 = tmp2.rename(columns={'latDeg_gt':'latDeg', 'lngDeg_gt':'lngDeg'})
        tmp2['phoneName'] = tmp2['phoneName'] + '_GT'
        tmp3 = self.bl[self.bl['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp3['phoneName'] = tmp3['phoneName'] + '_BL'
        
        if show_gt:
            tmp = tmp.append(tmp2)
        if show_bl:
            tmp = tmp.append(tmp3)
        visualize_collection(tmp, collection)

In [13]:
def get_data():
    base_train = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
    base_test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')
    sample_sub = pd.read_csv(INPUT + '/' + 'sample_submission.csv')
    ground_truth = get_ground_truth()
    return base_train, base_test, sample_sub, ground_truth

In [14]:
def update_baseline(df, rb):
    rb = rb.rename(columns={'latDeg':'latDeg_rb', 'lngDeg':'lngDeg_rb'})
    df = df.merge(rb[['millisSinceGpsEpoch', 'phone', 'latDeg_rb', 'lngDeg_rb']], on=['millisSinceGpsEpoch', 'phone'], how='left')

    idx = df[~df['latDeg_rb'].isnull()].index
    df.loc[idx, 'latDeg'] = df.loc[idx, 'latDeg_rb']
    df.loc[idx, 'lngDeg'] = df.loc[idx, 'lngDeg_rb']
    
    df.drop(columns=['latDeg_rb', 'lngDeg_rb'], inplace=True)
    
    return df

In [15]:
def err_map(df_, output_dir):
    df = df_.copy()
    gt = pd.read_csv(INPUT + '/prep/ground_truth_train.csv')
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt[['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'latDeg_gt', 'lngDeg_gt', 'speedMps', 'courseDegree']], on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    df['err'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_gt'], df['lngDeg_gt'])
    
    os.makedirs(output_dir, exist_ok=True)
    for phone in df['phone'].unique():
        tmp = df[df['phone']==phone].copy()
        fig, axes = plt.subplots(figsize=(30, 30))
        sns.scatterplot(x='latDeg', y='lngDeg', hue='err', data=tmp, size='err', s=100, alpha=0.8, lw=0, ax=axes)
        axes.invert_xaxis()
        fig.suptitle(phone, fontsize=16)
        fig.savefig(f'{output_dir}/{phone}.png')
        plt.close()

In [16]:
def err_transition(df_, output_dir):
    df = df_.copy()
    gt = pd.read_csv(INPUT + '/prep/ground_truth_train.csv')
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt[['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'latDeg_gt', 'lngDeg_gt', 'speedMps', 'courseDegree']], on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    df['err'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_gt'], df['lngDeg_gt'])
    df['millis_diff'] = df['millisSinceGpsEpoch'] - df.groupby('phone')['millisSinceGpsEpoch'].shift(1)
    
    os.makedirs(output_dir, exist_ok=True)
    for phone in df['phone'].unique():
        tmp = df[df['phone']==phone].copy()
        fig, axes = plt.subplots(figsize=(40, 20), nrows=4,sharex=True)
        axes[0].plot(tmp['millisSinceGpsEpoch'], tmp['speedMps'], label='speedMps')
        axes[0].legend(loc='upper right')
        axes[0].grid(color='g', linestyle=':', linewidth=0.3)

        axes[1].plot(tmp['millisSinceGpsEpoch'], tmp['courseDegree'], label='courseDegree')
        axes[1].legend(loc='upper right')
        axes[1].grid(color='g', linestyle=':', linewidth=0.3)

        axes[2].plot(tmp['millisSinceGpsEpoch'], tmp['millis_diff'], label='millis_diff')
        axes[2].legend(loc='upper right')
        axes[2].grid(color='g', linestyle=':', linewidth=0.3)

        axes[3].plot(tmp['millisSinceGpsEpoch'], tmp['err'], label='err')
        axes[3].legend(loc='upper right')
        axes[3].grid(color='g', linestyle=':', linewidth=0.3)

        fig.suptitle(phone, fontsize=16)
        fig.savefig(f'{output_dir}/{phone}.png')
        plt.close()

In [17]:
def add_path_no(df):
    df['millis_diff'] = df['millisSinceGpsEpoch'] - df.groupby('phone')['millisSinceGpsEpoch'].shift(1)
    head_idx = df.groupby('phone').head(1).index
    df.loc[head_idx, 'path_no'] = 1
    df.loc[df['millis_diff'] > 1500, 'path_no'] = 1
    df['path_no'] = df['path_no'].cumsum().fillna(method='ffill')
    return df

# 自前再構成baselineに更新

In [18]:
train, test, sub, gt = get_data()

In [19]:
train = add_path_no(train)

In [20]:
result = train_result(train[train['collectionName'].isin(target)])
print(result.score)
display(result.err)

4.0212605327241775


,percentile50,percentile95,p50_p90_mean
phone,,,
2020-05-14-US-MTV-1_Pixel4,1.313621,2.887582,2.100602
2020-05-14-US-MTV-1_Pixel4XLModded,1.987538,4.352359,3.169948
2020-05-14-US-MTV-2_Pixel4,1.367776,2.799897,2.083836
2020-05-14-US-MTV-2_Pixel4XLModded,3.357252,11.367965,7.362609
2020-05-21-US-MTV-1_Pixel4,1.810603,4.670839,3.240721
2020-05-21-US-MTV-2_Pixel4,1.006633,3.232224,2.119429
2020-05-21-US-MTV-2_Pixel4XL,1.160861,3.393783,2.277322
2020-05-29-US-MTV-1_Pixel4,2.627895,4.710244,3.669069
2020-05-29-US-MTV-1_Pixel4XL,2.326845,3.836519,3.081682


In [21]:
train = update_baseline(train, pd.read_csv('../output/prep/baseline_g1_v001/result.csv'))
train = update_baseline(train, pd.read_csv('../output/prep/baseline_g2_v003/result.csv'))
train = update_baseline(train, pd.read_csv('../output/prep/baseline_g3_v003/result.csv'))
train = update_baseline(train, pd.read_csv('../output/prep/baseline_g4_v002/result.csv'))

In [22]:
result = train_result(train[train['collectionName'].isin(target)])
print(result.score)
display(result.err)

3.839905340819292


,percentile50,percentile95,p50_p90_mean
phone,,,
2020-05-14-US-MTV-1_Pixel4,1.116276,2.400690,1.758483
2020-05-14-US-MTV-1_Pixel4XLModded,1.691268,3.686179,2.688723
2020-05-14-US-MTV-2_Pixel4,1.523077,2.728798,2.125938
2020-05-14-US-MTV-2_Pixel4XLModded,3.379125,10.093809,6.736467
2020-05-21-US-MTV-1_Pixel4,1.943373,4.794928,3.369151
2020-05-21-US-MTV-2_Pixel4,0.898226,2.804577,1.851401
2020-05-21-US-MTV-2_Pixel4XL,1.085159,3.437475,2.261317
2020-05-29-US-MTV-1_Pixel4,2.485724,4.288529,3.387127
2020-05-29-US-MTV-1_Pixel4XL,2.235506,3.395295,2.815400


# 相対移動距離をもとにした外れ値除去

In [23]:
def add_distance_diff(df):
    df['latDeg_prev'] = df['latDeg'].shift(1)
    df['latDeg_next'] = df['latDeg'].shift(-1)
    df['lngDeg_prev'] = df['lngDeg'].shift(1)
    df['lngDeg_next'] = df['lngDeg'].shift(-1)
    df['phone_prev'] = df['phone'].shift(1)
    df['phone_next'] = df['phone'].shift(-1)
    
    df['dist_prev'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_prev'], df['lngDeg_prev'])
    df['dist_next'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_next'], df['lngDeg_next'])
    
    df.loc[df['path_no']!=df['path_no'], ['latDeg_prev', 'lngDeg_prev', 'dist_prev']] = np.nan
    df.loc[df['path_no']!=df['path_no'], ['latDeg_next', 'lngDeg_next', 'dist_next']] = np.nan
    
    return df

In [24]:
# reject outlier
train = add_distance_diff(train)
train.loc[((train['dist_prev'] > ro_th) & (train['dist_next'] > ro_th)), ['latDeg', 'lngDeg']] = np.nan
train.to_csv(OUTPUT + '/train_ro.csv', index=False)

# ground_truthを基準にした外れ値除去

In [25]:
def get_osmnx_data():
    p = pathlib.Path(INPUT)
    files = list(p.glob('prep/osmnx/*.csv'))

    dfs = []
    for file in files:
        dfs.append(pd.read_csv(file))
    osmnx = pd.concat(dfs)

    return osmnx

In [26]:
def remove_based_on_gt(target_df, target_gt, th):
    osmnx_data = get_osmnx_data()
    target_gt = target_gt.append(osmnx_data)
    
    for idx in target_df.index:
        lat = target_df.at[idx, 'latDeg']
        lng = target_df.at[idx, 'lngDeg']
        collection = target_df.at[idx, 'collectionName']
        
        if collection in ['2021-04-22-US-SJC-1', '2021-04-29-US-SJC-2', '2021-04-28-US-SJC-1', '2021-04-22-US-SJC-2', '2021-04-29-US-SJC-3']:
            th = 3
        
        if np.isnan(lat):
            continue
        else:
            target_gt['latDeg_pred'] = lat
            target_gt['lngDeg_pred'] = lng
            target_gt['dist'] = calc_haversine(target_gt['latDeg'], target_gt['lngDeg'], target_gt['latDeg_pred'], target_gt['lngDeg_pred'])
            closest_dist = target_gt['dist'].min()
            if closest_dist > th:
                target_df.at[idx, 'latDeg'] = np.nan
                target_df.at[idx, 'lngDeg'] = np.nan
    return target_df

In [27]:
def interpolate_remove_point(df):
    out_df = pd.DataFrame()
    collections = df['collectionName'].unique()
    
    for collection in collections:
        tmp = df[df['collectionName']==collection].copy()
        tmp = tmp.sort_values('millisSinceGpsEpoch')
        tmp = tmp.reset_index().set_index('millisSinceGpsEpoch')
        tmp[['latDeg', 'lngDeg']] = tmp[['latDeg', 'lngDeg']].interpolate(method='index', limit_area='inside')
        tmp = tmp.sort_values('index')
        tmp = tmp.reset_index().set_index('index')
        out_df = out_df.append(tmp)
    return out_df

In [28]:
rog_df = train[train['collectionName'].isin(rog_target)].copy()
rog_gt = gt[gt['collectionName'].isin(rog_target)].copy()
rog_df = remove_based_on_gt(rog_df, rog_gt, rog_th)
train.loc[rog_df.index, ['latDeg', 'lngDeg']] = rog_df[['latDeg', 'lngDeg']]

In [29]:
train = interpolate_remove_point(train)

In [30]:
result = train_result(train[train['collectionName'].isin(target)])
print(result.score)
display(result.err)

err_map(train, f'{OUTPUT}/train/after_ro/map')
err_transition(train, f'{OUTPUT}/train/after_ro/transition')

3.686794379089488


,percentile50,percentile95,p50_p90_mean
phone,,,
2020-05-14-US-MTV-1_Pixel4,1.116276,2.400690,1.758483
2020-05-14-US-MTV-1_Pixel4XLModded,1.691268,3.686179,2.688723
2020-05-14-US-MTV-2_Pixel4,1.523077,2.728798,2.125938
2020-05-14-US-MTV-2_Pixel4XLModded,3.364616,9.856474,6.610545
2020-05-21-US-MTV-1_Pixel4,1.943373,4.794928,3.369151
2020-05-21-US-MTV-2_Pixel4,0.898226,2.804577,1.851401
2020-05-21-US-MTV-2_Pixel4XL,1.084187,3.346398,2.215292
2020-05-29-US-MTV-1_Pixel4,2.480581,4.232507,3.356544
2020-05-29-US-MTV-1_Pixel4XL,2.235506,3.380992,2.808249


# kalmanフィルタ

In [31]:
T = 1.0
state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0], [0, 1, 0, T, 0, 0.5 * T ** 2], [0, 0, 1, 0, T, 0],
                             [0, 0, 0, 1, 0, T], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
observation_noise = np.diag([5e-5, 5e-5]) + np.ones((2, 2)) * 1e-9

kf = simdkalman.KalmanFilter(
        state_transition = state_transition,
        process_noise = process_noise,
        observation_model = observation_model,
        observation_noise = observation_noise)

def apply_kf_smoothing(df, kf_=kf):
    unique_paths = df['path_no'].unique()
    for path in unique_paths:
        idx = df[df['path_no']==path].index
        if len(idx) > 1:
            data = df.loc[idx][['latDeg', 'lngDeg']].to_numpy()
            data = data.reshape(1, len(data), 2)
            smoothed = kf_.smooth(data)
            df.loc[idx, 'latDeg'] = smoothed.states.mean[0, :, 0]
            df.loc[idx, 'lngDeg'] = smoothed.states.mean[0, :, 1]
    return df

In [32]:
train = apply_kf_smoothing(train)

In [33]:
result = train_result(train[train['collectionName'].isin(target)])
print(result.score)
display(result.err)

err_map(train, f'{OUTPUT}/train/after_kf/map')
err_transition(train, f'{OUTPUT}/train/after_kf/transition')

3.1560704171005423


,percentile50,percentile95,p50_p90_mean
phone,,,
2020-05-14-US-MTV-1_Pixel4,1.095690,2.078749,1.587219
2020-05-14-US-MTV-1_Pixel4XLModded,1.693251,3.083921,2.388586
2020-05-14-US-MTV-2_Pixel4,1.492479,2.688265,2.090372
2020-05-14-US-MTV-2_Pixel4XLModded,2.333526,6.608347,4.470937
2020-05-21-US-MTV-1_Pixel4,1.917192,4.405775,3.161483
2020-05-21-US-MTV-2_Pixel4,0.836022,2.615265,1.725643
2020-05-21-US-MTV-2_Pixel4XL,1.031630,3.192393,2.112012
2020-05-29-US-MTV-1_Pixel4,2.403147,3.822418,3.112782
2020-05-29-US-MTV-1_Pixel4XL,2.219104,3.328389,2.773747


# speed0の処理

In [34]:
def sp0_process(df, sp0_result):
    df = df.merge(sp0_result, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    df['group'] = df.groupby('phone').cumcount()
    df.loc[(df['group']>0)&(df['isSpeed0']==1),'group'] = np.nan
    df['group'] = df['group'].fillna(method='ffill')
    df[['latDeg', 'lngDeg']] = df.groupby(['phone', 'group'])[['latDeg','lngDeg']].transform('mean')
    return df

In [35]:
train_sp0_pred = pd.read_csv('../output/prep/speed0_pred_v001/train_sp0_pred.csv')
train = sp0_process(train, train_sp0_pred)
train.to_csv(OUTPUT + '/train_ro_rog_kf_sp0.csv', index=False)

In [36]:
result = train_result(train[train['collectionName'].isin(target)])
print(result.score)
display(result.err)

err_map(train, f'{OUTPUT}/train/after_sp0/map')
err_transition(train, f'{OUTPUT}/train/after_sp0/transition')

2.9346468449795777


,percentile50,percentile95,p50_p90_mean
phone,,,
2020-05-14-US-MTV-1_Pixel4,1.083894,1.989597,1.536746
2020-05-14-US-MTV-1_Pixel4XLModded,1.691526,2.928716,2.310121
2020-05-14-US-MTV-2_Pixel4,1.488251,2.503225,1.995738
2020-05-14-US-MTV-2_Pixel4XLModded,2.308673,6.700052,4.504363
2020-05-21-US-MTV-1_Pixel4,1.861202,3.567697,2.714450
2020-05-21-US-MTV-2_Pixel4,0.759596,2.345042,1.552319
2020-05-21-US-MTV-2_Pixel4XL,1.044972,2.718381,1.881676
2020-05-29-US-MTV-1_Pixel4,2.347207,3.553755,2.950481
2020-05-29-US-MTV-1_Pixel4XL,2.223404,3.380523,2.801964


# phones_weighted_mean

In [37]:
train_ = train[train['collectionName'].isin(target)].copy()

In [38]:
def calc_mean_pred(df, ws):
    '''
    Generate interpolated lat,lng values for different phone times in the same collection.
    '''
    time_list = df[['collectionName', 'millisSinceGpsEpoch']].drop_duplicates()
    phone_list = df[['collectionName', 'phoneName']].drop_duplicates()
    tmp = time_list.merge(phone_list, on='collectionName', how='outer')
    tmp['phone'] = tmp['collectionName'] + '_' + tmp['phoneName']
    tmp = tmp.merge(df, on=['collectionName', 'phoneName', 'phone', 'millisSinceGpsEpoch'], how='left')
    
    lerp_df = pd.DataFrame()
    for phone in tmp['phone'].unique():
        lerp_tmp = tmp[tmp['phone']==phone].copy()
        lerp_tmp = lerp_tmp.sort_values('millisSinceGpsEpoch')
        
        lerp_tmp.loc[~lerp_tmp['latDeg'].isnull(), 'ms'] = lerp_tmp.loc[~lerp_tmp['latDeg'].isnull(), 'millisSinceGpsEpoch']
        lerp_tmp['ms'] = lerp_tmp['ms'].fillna(method='ffill')
        lerp_tmp['msd'] = lerp_tmp['millisSinceGpsEpoch'] - lerp_tmp['ms']
        lerp_tmp = lerp_tmp[lerp_tmp['msd']<1500].copy()
        
        lerp_tmp = lerp_tmp.set_index('millisSinceGpsEpoch')
        lerp_tmp[['latDeg', 'lngDeg']] = lerp_tmp[['latDeg', 'lngDeg']].interpolate(method='index', limit_area='inside')
        lerp_tmp = lerp_tmp.reset_index()
        lerp_df = lerp_df.append(lerp_tmp)
    
    lerp_df['w'] = lerp_df['phoneName'].replace(ws)
    lerp_df.loc[lerp_df['latDeg'].isnull(), 'w'] = np.nan
    lerp_df['w_sum'] = lerp_df.groupby(['collectionName', 'millisSinceGpsEpoch'])['w'].transform('sum')
    lerp_df['w'] = lerp_df['w'] / lerp_df['w_sum']
    lerp_df['latDeg'] = lerp_df['latDeg'] * lerp_df['w']
    lerp_df['lngDeg'] = lerp_df['lngDeg'] * lerp_df['w']
    mean_pred = lerp_df.groupby(['collectionName', 'millisSinceGpsEpoch'])['latDeg', 'lngDeg'].sum().reset_index()
    
    output_df = df[['collectionName', 'phoneName', 'millisSinceGpsEpoch']].copy()
    output_df = output_df.merge(mean_pred[['collectionName', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']], on=['collectionName', 'millisSinceGpsEpoch'], how='left')
    return output_df

In [39]:
def phones_mean_objective(trial):
    Pixel4 = trial.suggest_int('Pixel4', 1, 10)
    Pixel4XLModded = trial.suggest_int('Pixel4XLModded', 1, 10)
    Pixel4XL = trial.suggest_int('Pixel4XL', 1, 10)
    Mi8 = trial.suggest_int('Mi8', 1, 10)
    Pixel4Modded = trial.suggest_int('Pixel4Modded', 1, 10)
    Pixel5 = trial.suggest_int('Pixel5', 1, 10)
    SamsungS20Ultra = trial.suggest_int('SamsungS20Ultra', 1, 10)
    
    ws = {'Pixel4':Pixel4, 'Pixel4XLModded':Pixel4XLModded, 'Pixel4XL':Pixel4XL, 'Mi8':Mi8, 'Pixel4Modded':Pixel4Modded,
          'Pixel5':Pixel5, 'SamsungS20Ultra':SamsungS20Ultra}
    
    out = calc_mean_pred(train_, ws)
    score = train_result(out).score
    
    return score

In [40]:
study = optuna.create_study()
study.optimize(phones_mean_objective, n_trials=100)
phones_mean_weights = study.best_params
print(phones_mean_weights)

[I 2021-07-26 22:10:30,317] A new study created in memory with name: no-name-ef61f2b6-027b-42c2-bb35-f8c007c2e5f2
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
[I 2021-07-26 22:10:35,669] Trial 0 finished with value: 2.5502380391351034 and parameters: {'Pixel4': 3, 'Pixel4XLModded': 8, 'Pixel4XL': 7, 'Mi8': 7, 'Pixel4Modded': 4, 'Pixel5': 5, 'SamsungS20Ultra': 6}. Best is trial 0 with value: 2.5502380391351034.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
[I 2021-07-26 22:10:40,987] Trial 1 finished with value: 2.5414858872376183 and parameters: {'Pixel4': 1, 'Pixel4XLModded': 3, 'Pixel4XL': 4, 'Mi8': 1, 'Pixel4Modded': 9, 'Pixel5': 4, 'SamsungS20Ultra': 8}. Best is trial 1 with value: 2.54148588723761

{'Pixel4': 4, 'Pixel4XLModded': 2, 'Pixel4XL': 2, 'Mi8': 4, 'Pixel4Modded': 9, 'Pixel5': 2, 'SamsungS20Ultra': 1}


In [41]:
train = calc_mean_pred(train, phones_mean_weights)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [42]:
result = train_result(train[train['collectionName'].isin(target)])
print(result.score)
display(result.err)

train['phone'] = train['collectionName'] + '_' + train['phoneName']
err_map(train, f'{OUTPUT}/train/after_pm/map')
err_transition(train, f'{OUTPUT}/train/after_pm/transition')

2.3485683991671813


,percentile50,percentile95,p50_p90_mean
phone,,,
2020-05-14-US-MTV-1_Pixel4,1.187425,1.934705,1.561065
2020-05-14-US-MTV-1_Pixel4XLModded,1.102618,1.854749,1.478683
2020-05-14-US-MTV-2_Pixel4,1.537665,2.667499,2.102582
2020-05-14-US-MTV-2_Pixel4XLModded,1.612621,3.351757,2.482189
2020-05-21-US-MTV-1_Pixel4,1.861202,3.567697,2.714450
2020-05-21-US-MTV-2_Pixel4,0.759474,2.437003,1.598238
2020-05-21-US-MTV-2_Pixel4XL,0.732640,2.251284,1.491962
2020-05-29-US-MTV-1_Pixel4,2.169794,3.033282,2.601538
2020-05-29-US-MTV-1_Pixel4XL,2.122699,3.027538,2.575119


# position_shift

In [43]:
def WGS84_to_ECEF(lat, lon, alt):
    # convert to radians
    rad_lat = lat * (np.pi / 180.0)
    rad_lon = lon * (np.pi / 180.0)
    a    = 6378137.0
    # f is the flattening factor
    finv = 298.257223563
    f = 1 / finv   
    # e is the eccentricity
    e2 = 1 - (1 - f) * (1 - f)    
    # N is the radius of curvature in the prime vertical
    N = a / np.sqrt(1 - e2 * np.sin(rad_lat) * np.sin(rad_lat))
    x = (N + alt) * np.cos(rad_lat) * np.cos(rad_lon)
    y = (N + alt) * np.cos(rad_lat) * np.sin(rad_lon)
    z = (N * (1 - e2) + alt)        * np.sin(rad_lat)
    return x, y, z

transformer = pyproj.Transformer.from_crs(
    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},)

def ECEF_to_WGS84(x,y,z):
    lon, lat, alt = transformer.transform(x,y,z,radians=False)
    return lon, lat, alt
msge = 'millisSinceGpsEpoch'

In [44]:
def position_shift(df,a):

    d = df.copy()
    d['heightAboveWgs84EllipsoidM'] = 63.5
    d['x'], d['y'], d['z'] = zip(*d.apply(lambda x: WGS84_to_ECEF(x.latDeg, x.lngDeg, x.heightAboveWgs84EllipsoidM), axis=1))

    #a = -0.2
    d.sort_values(['phone', msge], inplace=True)
    for fi in ['x','y','z']:
        d[[fi+'p']] = d[fi].shift().where(d['phone'].eq(d['phone'].shift()))
        d[[fi+'diff']] = d[fi]-d[fi+'p']
    #d[['yp']] = d['y'].shift().where(d['phone'].eq(d['phone'].shift()))
    d[['dist']] = np.sqrt(d['xdiff']**2 + d['ydiff']**2+ d['zdiff']**2)
    for fi in ['x','y','z']:
        d[[fi+'new']] = d[fi+'p'] + d[fi+'diff']*(1-a/d['dist'])
    lng, lat, alt = ECEF_to_WGS84(d['xnew'].values,d['ynew'].values,d['znew'].values)
    
    lng[np.isnan(lng)] = d.loc[np.isnan(lng),'lngDeg']
    lat[np.isnan(lat)] = d.loc[np.isnan(lat),'latDeg']
    d['latDeg'] = lat
    d['lngDeg'] = lng
    
    return d 

def objective(trial):
    a = trial.suggest_uniform('a', -1, 1)
    score = get_train_score(position_shift(train, a),gt)
    return score

In [45]:
study = optuna.create_study()
study.optimize(objective, n_trials=30)
opt_a = study.best_params['a']
print(opt_a)

[I 2021-07-26 22:20:30,789] A new study created in memory with name: no-name-804d81e3-e877-4a9c-b110-3e07e08f89cf
[I 2021-07-26 22:20:35,878] Trial 0 finished with value: 2.998744100519799 and parameters: {'a': -0.03021928208980218}. Best is trial 0 with value: 2.998744100519799.
[I 2021-07-26 22:20:39,728] Trial 1 finished with value: 2.965901453054599 and parameters: {'a': 0.6699918769826034}. Best is trial 1 with value: 2.965901453054599.
[I 2021-07-26 22:20:43,575] Trial 2 finished with value: 2.9730677289245038 and parameters: {'a': 0.748350920188116}. Best is trial 1 with value: 2.965901453054599.
[I 2021-07-26 22:20:47,495] Trial 3 finished with value: 3.286523256690787 and parameters: {'a': -0.8025019602734909}. Best is trial 1 with value: 2.965901453054599.
[I 2021-07-26 22:20:51,345] Trial 4 finished with value: 3.2726948449604674 and parameters: {'a': -0.77561551785128}. Best is trial 1 with value: 2.965901453054599.
[I 2021-07-26 22:20:55,161] Trial 5 finished with value: 2

0.3964282956068143


In [46]:
train = position_shift(train, opt_a)
train.to_csv(OUTPUT + '/train_ro_rog_kf_sp0_pm_ps.csv', index=False)

In [47]:
result = train_result(train[train['collectionName'].isin(target)])
print(result.score)
display(result.err)

train['phone'] = train['collectionName'] + '_' + train['phoneName']
err_map(train, f'{OUTPUT}/train/after_pm/map')
err_transition(train, f'{OUTPUT}/train/after_ps/transition')

2.31259064857877


,percentile50,percentile95,p50_p90_mean
phone,,,
2020-05-14-US-MTV-1_Pixel4,0.980688,1.756208,1.368448
2020-05-14-US-MTV-1_Pixel4XLModded,0.904140,1.641690,1.272915
2020-05-14-US-MTV-2_Pixel4,1.834464,2.854490,2.344477
2020-05-14-US-MTV-2_Pixel4XLModded,1.806412,3.524146,2.665279
2020-05-21-US-MTV-1_Pixel4,1.993015,3.567715,2.780365
2020-05-21-US-MTV-2_Pixel4,0.695229,2.447490,1.571359
2020-05-21-US-MTV-2_Pixel4XL,0.646868,2.222169,1.434518
2020-05-29-US-MTV-1_Pixel4,1.808491,2.657217,2.232854
2020-05-29-US-MTV-1_Pixel4XL,1.792718,2.657615,2.225167


In [48]:
train = train[train['collectionName'].isin(target)].copy()

# trainの結果確認

In [49]:
result = train_result(train)
print(result.score)
display(result.err)

2.31259064857877


,percentile50,percentile95,p50_p90_mean
phone,,,
2020-05-14-US-MTV-1_Pixel4,0.980688,1.756208,1.368448
2020-05-14-US-MTV-1_Pixel4XLModded,0.904140,1.641690,1.272915
2020-05-14-US-MTV-2_Pixel4,1.834464,2.854490,2.344477
2020-05-14-US-MTV-2_Pixel4XLModded,1.806412,3.524146,2.665279
2020-05-21-US-MTV-1_Pixel4,1.993015,3.567715,2.780365
2020-05-21-US-MTV-2_Pixel4,0.695229,2.447490,1.571359
2020-05-21-US-MTV-2_Pixel4XL,0.646868,2.222169,1.434518
2020-05-29-US-MTV-1_Pixel4,1.808491,2.657217,2.232854
2020-05-29-US-MTV-1_Pixel4XL,1.792718,2.657615,2.225167


In [50]:
train.to_csv(OUTPUT + f'/{nb_name}_train.csv', index=False)

# make_sub

In [51]:
test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')

In [52]:
test = add_path_no(test)

In [53]:
test = update_baseline(test, pd.read_csv('../output/prep/baseline_g1_v001/result_test.csv'))
test = update_baseline(test, pd.read_csv('../output/prep/baseline_g2_v003/result_test.csv'))
test = update_baseline(test, pd.read_csv('../output/prep/baseline_g3_v003/result_test.csv'))
test = update_baseline(test, pd.read_csv('../output/prep/baseline_g4_v002/result_test.csv'))

In [54]:
test = add_distance_diff(test)
test.loc[((test['dist_prev'] > ro_th) & (test['dist_next'] > ro_th)), ['latDeg', 'lngDeg']] = np.nan

In [55]:
rog_df = test[test['collectionName'].isin(rog_target)].copy()
rog_gt = gt[gt['collectionName'].isin(rog_target)].copy()
rog_df = remove_based_on_gt(rog_df, rog_gt, rog_th)
rog_df.to_csv(OUTPUT + '/test_rog.csv', index=False)
test.loc[rog_df.index, ['latDeg', 'lngDeg']] = rog_df[['latDeg', 'lngDeg']]

In [56]:
test = interpolate_remove_point(test)

In [57]:
test = apply_kf_smoothing(test)

In [58]:
test_sp0_pred = pd.read_csv('../output/prep/speed0_pred_v001/test_sp0_pred.csv')
test = sp0_process(test, test_sp0_pred)

In [59]:
test = calc_mean_pred(test, phones_mean_weights)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [60]:
test['phone'] = test['collectionName'] + '_' + test['phoneName']

In [61]:
test = position_shift(test, opt_a)

In [62]:
test = test[test['collectionName'].isin(target)]

In [63]:
test.to_csv(OUTPUT + f'/{nb_name}_test.csv', index=False)

In [64]:
for phone in test['phone'].unique():
    tmp = test[test['phone']==phone].copy()
    fig, axes = plt.subplots(figsize=(30, 30))
    sns.scatterplot(x='latDeg', y='lngDeg', data=tmp, alpha=0.8, lw=0, ax=axes)
    axes.invert_xaxis()
    fig.suptitle(phone, fontsize=16)
    fig.savefig(f'{OUTPUT}/{phone}.png')
    plt.close()